# 06 — Keyphrases (YAKE + Fallback)

In diesem Notebook extrahieren wir Keyphrases aus dem Demo‑Korpus.

- **YAKE** als primäre Methode (Sprache: Deutsch)
- **TF‑IDF Fallback** (2–3‑Gramme), falls `yake` nicht verfügbar ist
- Robuste Pfadlogik für `./data` bzw. `../data`
- Export als **JSON** (Liste je Post) und **CSV** (eine Phrase pro Zeile)

> Tipp: Falls `yake` fehlt, kannst du im Terminal deiner venv installieren:
> `python -m pip install yake`


In [7]:
# 🔧 Robustes Setup
from pathlib import Path
import json

# Finde den /data-Ordner, bevorzugt den, der die Datei enthält
CWD = Path.cwd().resolve()
candidates = [
    CWD / "data",
    CWD.parent / "data",
]

DATA = None
for d in candidates:
    if (d / "sample_corpus.json").exists():
        DATA = d
        break
if DATA is None:
    raise FileNotFoundError("sample_corpus.json nicht gefunden unter ./data oder ../data")

SAMPLE = DATA / "sample_corpus.json"
texts = json.loads(SAMPLE.read_text(encoding="utf-8"))
print(f"📚 {len(texts)} Texte aus: {SAMPLE}")

📚 10 Texte aus: /Users/sm/Documents/mixing-forum-analyzer/data/sample_corpus.json


In [8]:
# YAKE importieren – wenn nicht vorhanden, setzen wir einen Fallback-Flag
try:
    import yake  # type: ignore
    HAVE_YAKE = True
    YAKE_IMPORT_ERROR = None
except Exception as e:
    HAVE_YAKE = False
    YAKE_IMPORT_ERROR = str(e)
print("YAKE:", "ok" if HAVE_YAKE else f"nicht verfügbar ({YAKE_IMPORT_ERROR})")

YAKE: ok


In [9]:
from typing import List, Dict
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

def extract_keyphrases_yake(texts: List[str], lang: str = "de", n: int = 3, topk: int = 10) -> List[Dict]:
    """YAKE: Top-`topk` Phrasen pro Text."""
    kw = yake.KeywordExtractor(lan=lang, n=n, top=topk)
    return [{"post_id": i, "phrases": [k for k, _ in kw.extract_keywords(t)]} for i, t in enumerate(texts)]

def extract_keyphrases_tfidf(texts: List[str], ngram_range=(2, 3), topk: int = 10) -> List[Dict]:
    """Fallback: TF-IDF Rangfolge für 2–3-Gramme pro Dokument."""
    vec = TfidfVectorizer(lowercase=True, ngram_range=ngram_range, min_df=1)
    X = vec.fit_transform(texts)
    feats = np.array(vec.get_feature_names_out())
    results: List[Dict] = []
    for i in range(X.shape[0]):
        row = X.getrow(i)
        if row.nnz == 0:
            results.append({"post_id": i, "phrases": []})
            continue
        # Indizes der höchsten TF-IDF Gewichte pro Dokument
        order = row.indices[np.argsort(row.data)[::-1]]
        phrases = feats[order][:topk].tolist()
        results.append({"post_id": i, "phrases": phrases})
    return results

In [10]:
if HAVE_YAKE:
    keyphrases = extract_keyphrases_yake(texts, lang="de", n=3, topk=10)
else:
    print("⚠️ YAKE nicht verfügbar – nutze TF-IDF Fallback (2–3-Gramme).")
    keyphrases = extract_keyphrases_tfidf(texts, ngram_range=(2, 3), topk=10)

len(keyphrases), keyphrases[:2]

(10,
 [{'post_id': 0,
   'phrases': ['pumpt im Mix',
    'Kickdrum pumpt',
    'Snare wirkt',
    'wirkt zu dünn',
    'Mix',
    'Kickdrum',
    'Snare',
    'dünn',
    'pumpt',
    'wirkt']},
  {'post_id': 1,
   'phrases': ['kHz Bereich',
    'Vocals sitzen',
    'Bereich',
    'Präsenz',
    'Vocals',
    'hinten',
    'kHz',
    'sitzen']}])

In [11]:
import pandas as pd

OUT_JSON = DATA / "keyphrases.json"
OUT_CSV  = DATA / "keyphrases.csv"

# Sicherstellen, dass 'keyphrases' existiert
if 'keyphrases' not in globals():
    raise RuntimeError("Keine Variable 'keyphrases' gefunden – bitte erst die Extraktion-Zelle ausführen.")

# JSON (Liste pro Post)
OUT_JSON.write_text(json.dumps(keyphrases, ensure_ascii=False, indent=2), encoding="utf-8")

# CSV (eine Phrase pro Zeile)
rows = [{"post_id": item["post_id"], "phrase": p} for item in keyphrases for p in item.get("phrases", [])]
pd.DataFrame(rows).to_csv(OUT_CSV, index=False)

print("✅ Gespeichert:", OUT_JSON)
print("✅ Gespeichert:", OUT_CSV)

# Vorschau
pd.read_json(OUT_JSON).head()

✅ Gespeichert: /Users/sm/Documents/mixing-forum-analyzer/data/keyphrases.json
✅ Gespeichert: /Users/sm/Documents/mixing-forum-analyzer/data/keyphrases.csv


,post_id,phrases
0,0,"[pumpt im Mix, Kickdrum pumpt, Snare wirkt, wi..."
1,1,"[kHz Bereich, Vocals sitzen, Bereich, Präsenz,..."
2,2,"[Snare klingt trocken, Snare klingt, Raumantei..."
3,3,"[Bassdrum und Kickdrum, brauche mehr Punch, Pu..."
4,4,"[Low-Pass könnte helfen, sanfter Low-Pass, sch..."
